In [1]:
# pip install -U kaleido
# pip install --upgrade scikit-learn


In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio


import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



Cannot find header.dxf (GDAL_DATA is not defined)


In [3]:
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-stac_sotc/catalog.json" # cloud API
    #os.path.join(os.path.dirname(cwd), "stac_folder", "current", "catalog.json") # local clone 
)
catalog

<Client id=gca-catalog>

In [4]:
list(catalog.get_children())

[<CollectionClient id=shore_mon_fut>,
 <CollectionClient id=sed_class>,
 <CollectionClient id=shore_mon>,
 <CollectionClient id=world_pop>,
 <CollectionClient id=world_gdp>,
 <CollectionClient id=shore_mon_hr>,
 <CollectionClient id=shore_mon_drivers>,
 <CollectionClient id=esl_gwl>,
 <CollectionClient id=sub_threat>,
 <CollectionClient id=Exp_world_pop>,
 <CollectionClient id=Exp_world_pop_parquet>,
 <CollectionClient id=Haz-ESL>,
 <CollectionClient id=Vul-Inf_Mort>,
 <CollectionClient id=Haz-GCM_90-100>,
 <CollectionClient id=Exp-NS_Slopes>,
 <CollectionClient id=Vul_MYS>,
 <CollectionClient id=Vul_RS>,
 <CollectionClient id=Haz-GCM_40-50>,
 <CollectionClient id=Haz-ERA5_04-14>,
 <CollectionClient id=Vul-GDP_Cap>,
 <CollectionClient id=Haz-GCM_04-14>,
 <CollectionClient id=Haz-Land_Sub_2010_COGs>,
 <CollectionClient id=Haz-Land_Sub_2040_COGs>,
 <CollectionClient id=Vul-Roads_Density_COGs>,
 <CollectionClient id=Exp-Delta_DEM_COGs>]

In [5]:
template_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\Arjen_Vector_Template.csv")
Hazard_map=template_df
Hazard_map = gpd.GeoDataFrame(
    Hazard_map, geometry=gpd.points_from_xy(Hazard_map.Intersect_lon, Hazard_map.Intersect_lat), crs="EPSG:4326"
)
Hazard_map=Hazard_map.drop(columns=["Start_lon","Start_lat","End_lon","End_lat","Intersect_lon","Intersect_lat"])
Hazard_map

,transect_id,country_id,continent,country_name,geometry
0,BOX_028_183_0,CHL,South America,Chile,POINT (-74.39097 -50.38256)
1,BOX_028_183_1,CHL,South America,Chile,POINT (-74.38713 -50.38404)
2,BOX_028_183_2,CHL,South America,Chile,POINT (-74.38328 -50.38553)
3,BOX_028_183_3,CHL,South America,Chile,POINT (-74.37952 -50.38708)
4,BOX_028_183_4,CHL,South America,Chile,POINT (-74.37599 -50.38885)
...,...,...,...,...,...
1739821,BOX_211_067_149,RUS,Europe,Russia,POINT (39.93520 64.69835)
1739822,BOX_211_067_150,RUS,Europe,Russia,POINT (39.93884 64.69947)
1739823,BOX_211_067_151,RUS,Europe,Russia,POINT (39.94200 64.70083)
1739824,BOX_211_067_152,RUS,Europe,Russia,POINT (39.94470 64.70236)


In [6]:
IPCC=gpd.read_file(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\IPCC-WGI-reference-regions-v4.gpkg")
IPCC.head()

,Continent,Type,Name,Acronym,geometry
0,POLAR,Land,Greenland/Iceland,GIC,"MULTIPOLYGON (((-10.00000 62.00000, -10.43750 ..."
1,NORTH-AMERICA,Land,N.W.North-America,NWN,"MULTIPOLYGON (((-105.00000 50.00000, -105.4386..."
2,NORTH-AMERICA,Land,N.E.North-America,NEN,"MULTIPOLYGON (((-50.00000 50.00000, -50.44000 ..."
3,NORTH-AMERICA,Land,W.North-America,WNA,"MULTIPOLYGON (((-130.00000 50.00000, -129.5614..."
4,NORTH-AMERICA,Land,C.North-America,CNA,"MULTIPOLYGON (((-90.00000 50.00000, -90.00000 ..."


In [7]:
region_acronym= "CAR"
region_gdf=IPCC.loc[IPCC["Acronym"]==region_acronym,:]
# region_gdf=region_gdf.iloc[:,-1:]
region_gdf


,Continent,Type,Name,Acronym,geometry
8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,"MULTIPOLYGON (((-75.00000 12.00000, -75.32609 ..."


In [8]:
Hazard_map=Hazard_map.sjoin(region_gdf)
Hazard_map=Hazard_map.iloc[:,0:5]
Hazard_map.index=range(0,len(Hazard_map.index))
Hazard_map

,transect_id,country_id,continent,country_name,geometry
0,BOX_116_353_2,COL,South America,Colombia,POINT (-71.87980 12.25068)
1,BOX_116_353_3,COL,South America,Colombia,POINT (-71.87698 12.24731)
2,BOX_116_353_4,COL,South America,Colombia,POINT (-71.87563 12.24314)
3,BOX_116_353_5,COL,South America,Colombia,POINT (-71.87523 12.23877)
4,BOX_116_353_6,COL,South America,Colombia,POINT (-71.87483 12.23440)
...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,POINT (-71.66819 17.96526)
37505,BOX_139_002_62,DOM,North America,Dominican Republic,POINT (-71.66623 17.96142)
37506,BOX_139_002_63,DOM,North America,Dominican Republic,POINT (-71.66458 17.95744)
37507,BOX_139_002_64,DOM,North America,Dominican Republic,POINT (-71.66294 17.95347)


In [9]:
# href = catalog.get_child("Vul_RS").assets["data"].href
# ds = xr.open_zarr(href)#,chunks={"nstations":1739919,"time":1})
# ds

In [10]:
# df["transect_id"] = df["transect_id"].apply(lambda x: x.decode('utf-8'))
# df["country"] = df["country"].apply(lambda x: x.decode('utf-8'))
# df

In [11]:
# df.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\RS\Caribbean_RS.csv")

In [12]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\03_Vulnerability\World Database on Protected Areas\World_Protected_Areas_Arjen_Vector_Template_dtype_correct.csv")
# df=df.drop(columns=['index_right'])
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Intersect_lon, df.Intersect_lat), crs="EPSG:4326"
)
data_points=data_points.sjoin(region_gdf)
print (data_points.columns)
data_points.index=range(0,len(data_points.index))
data_points

Index(['transect_id', 'country_id', 'continent', 'country_name', 'Start_lon',
       'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon', 'End_lat',
       'PA_DEF', 'NAME', 'ORIG_NAME', 'DESIG', 'DESIG_ENG', 'DESIG_TYPE',
       'IUCN_CAT', 'INT_CRIT', 'MARINE', 'REP_M_AREA', 'GIS_M_AREA',
       'REP_AREA', 'GIS_AREA', 'NO_TAKE', 'NO_TK_AREA', 'STATUS', 'STATUS_YR',
       'GOV_TYPE', 'OWN_TYPE', 'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID',
       'SUB_LOC', 'PARENT_ISO3', 'ISO3', 'SUPP_INFO', 'CONS_OBJ', 'geometry',
       'index_right', 'Continent', 'Type', 'Name', 'Acronym'],
      dtype='object')


,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,...,PARENT_ISO3,ISO3,SUPP_INFO,CONS_OBJ,geometry,index_right,Continent,Type,Name,Acronym
0,BOX_138_246_26,HTI,North America,Haiti,-72.027424,18.227615,-72.033644,18.221458,-72.039864,18.215302,...,HTI,HTI,Not Applicable,Not Applicable,POINT (-72.03364 18.22146),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
1,BOX_138_246_27,HTI,North America,Haiti,-72.024183,18.224661,-72.030403,18.218504,-72.036623,18.212347,...,HTI,HTI,Not Applicable,Not Applicable,POINT (-72.03040 18.21850),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
2,BOX_138_246_28,HTI,North America,Haiti,-72.021808,18.222651,-72.027027,18.215706,-72.032245,18.208760,...,HTI,HTI,Not Applicable,Not Applicable,POINT (-72.02703 18.21571),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
3,BOX_138_240_8,HTI,North America,Haiti,-71.983214,18.213256,-71.987795,18.205916,-71.992376,18.198576,...,HTI,HTI,Not Applicable,Not Applicable,POINT (-71.98780 18.20592),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
4,BOX_138_240_7,HTI,North America,Haiti,-71.987078,18.215432,-71.991659,18.208092,-71.996240,18.200752,...,HTI,HTI,Not Applicable,Not Applicable,POINT (-71.99166 18.20809),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_56,DOM,North America,Dominican Republic,-71.676025,17.987644,-71.682444,17.981666,-71.688863,17.975688,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,POINT (-71.68244 17.98167),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
37505,BOX_139_002_57,DOM,North America,Dominican Republic,-71.672883,17.984591,-71.679301,17.978613,-71.685720,17.972636,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,POINT (-71.67930 17.97861),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
37506,BOX_139_002_58,DOM,North America,Dominican Republic,-71.669412,17.981067,-71.676227,17.975501,-71.683043,17.969935,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,POINT (-71.67623 17.97550),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR
37507,BOX_139_002_59,DOM,North America,Dominican Republic,-71.666486,17.977826,-71.673302,17.972260,-71.680117,17.966693,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,POINT (-71.67330 17.97226),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR


In [13]:
data_points["STATUS"].unique()

array(['Designated', 'Proposed', 'Inscribed', 'Not Applicable'],
      dtype=object)

In [14]:
df["GOV_TYPE"].unique()

array(['Federal or national ministry or agency',
       'Non-profit organisations', 'Not Reported',
       'Government-delegated management',
       'Sub-national ministry or agency', 'Indigenous peoples',
       'Collaborative governance', 'Joint governance',
       'Local communities', 'For-profit organisations',
       'Individual landowners', 'Not Applicable'], dtype=object)

In [15]:

# Define a function to map status to RL1 value
def map_status_to_rl1(status):
    if status == "Proposed":
        return 5
    elif status == "Inscribed":
        return 4
    elif status == "Designated" or status == 'Adopted':
        return 3
    elif status == "Established":
        return 2
    elif status == 'Not Applicable' or status == 'Not Reported':
        return 1
    else:
        return np.nan

def map_gov_type_to_rl2(gov):
    if gov == 'Not Reported' or gov == 'Indigenous peoples' or gov == 'Local communities':
        return 5
    elif gov == 'Individual landowners' or gov == 'Sub-national ministry or agency' or gov == 'Non-profit organisations':
        return 4
    elif gov == 'Collaborative governance' or gov == 'Joint governance' or gov == 'For-profit organisations':
        return 3
    elif gov == 'Federal or national ministry or agency' or gov ==  'Government-delegated management':
        return 2
    elif gov == 'Not Applicable' or gov == 'Not Reported':
        return 1
    else:
        return np.nan

# Apply the mapping function to create the RL1 column
data_points["RL1"] = data_points["STATUS"].apply(lambda status: map_status_to_rl1(status))
data_points["RL2"] = data_points["GOV_TYPE"].apply(lambda gov: map_gov_type_to_rl2(gov))
# Create the Protected Area Vulnerability column by taking the maximum value between RL1 and RL2
data_points["World Protected Area Vulnerability"] = data_points.apply(lambda row: max(row["RL1"], row["RL2"]), axis=1)
# data_points["Vulnerability Cause"]
data_points

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,...,CONS_OBJ,geometry,index_right,Continent,Type,Name,Acronym,RL1,RL2,World Protected Area Vulnerability
0,BOX_138_246_26,HTI,North America,Haiti,-72.027424,18.227615,-72.033644,18.221458,-72.039864,18.215302,...,Not Applicable,POINT (-72.03364 18.22146),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,3,2,3
1,BOX_138_246_27,HTI,North America,Haiti,-72.024183,18.224661,-72.030403,18.218504,-72.036623,18.212347,...,Not Applicable,POINT (-72.03040 18.21850),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,3,2,3
2,BOX_138_246_28,HTI,North America,Haiti,-72.021808,18.222651,-72.027027,18.215706,-72.032245,18.208760,...,Not Applicable,POINT (-72.02703 18.21571),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,3,2,3
3,BOX_138_240_8,HTI,North America,Haiti,-71.983214,18.213256,-71.987795,18.205916,-71.992376,18.198576,...,Not Applicable,POINT (-71.98780 18.20592),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,3,2,3
4,BOX_138_240_7,HTI,North America,Haiti,-71.987078,18.215432,-71.991659,18.208092,-71.996240,18.200752,...,Not Applicable,POINT (-71.99166 18.20809),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_56,DOM,North America,Dominican Republic,-71.676025,17.987644,-71.682444,17.981666,-71.688863,17.975688,...,Not Applicable,POINT (-71.68244 17.98167),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,1,1,1
37505,BOX_139_002_57,DOM,North America,Dominican Republic,-71.672883,17.984591,-71.679301,17.978613,-71.685720,17.972636,...,Not Applicable,POINT (-71.67930 17.97861),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,1,1,1
37506,BOX_139_002_58,DOM,North America,Dominican Republic,-71.669412,17.981067,-71.676227,17.975501,-71.683043,17.969935,...,Not Applicable,POINT (-71.67623 17.97550),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,1,1,1
37507,BOX_139_002_59,DOM,North America,Dominican Republic,-71.666486,17.977826,-71.673302,17.972260,-71.680117,17.966693,...,Not Applicable,POINT (-71.67330 17.97226),8,CENTRAL-AMERICA,Land-Ocean,Caribbean,CAR,1,1,1


In [16]:
data_points.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\WPA\Caribbean_WPA.csv",index=False)

In [42]:
# Hazard_map["EWH_100yrRP"]=data_points["100_year_RP"]
Hazard_map_merged = pd.merge(left=Hazard_map, right=data_points, left_on="transect_id", right_on="transect_id")
Hazard_map["World Protected Area Vulnerability"]=Hazard_map_merged["World Protected Area Vulnerability"]
# Hazard_map["Vulnerability type"]=Hazard_map_merged["GOV_TYPE"]
Hazard_map.head()


,transect_id,country_id,continent,country_name,geometry,World Protected Area Vulnerability
0,BOX_116_353_2,COL,South America,Colombia,POINT (-71.87980 12.25068),3
1,BOX_116_353_3,COL,South America,Colombia,POINT (-71.87698 12.24731),3
2,BOX_116_353_4,COL,South America,Colombia,POINT (-71.87563 12.24314),3
3,BOX_116_353_5,COL,South America,Colombia,POINT (-71.87523 12.23877),3
4,BOX_116_353_6,COL,South America,Colombia,POINT (-71.87483 12.23440),3


In [44]:
import plotly.express as px
directory_path = r"C:\Users\rowe\Documents\GitHub\global-coastal-atlas\STAC\visualization\SOTC_transect_mapping_visual\Caribbean\Caribbean Plots"
lat = 18.1
lon = -77.3
view_width = 10

# # Filter dataframe within specified range
# filtered_df = df_final_merged[
#     (df_final_merged["Intersect_lon"] < lon + 18) & 
#     (df_final_merged["Intersect_lon"] > lon - 7.5) & 
#     (df_final_merged["Intersect_lat"] < lat + 9) & 
#     (df_final_merged["Intersect_lat"] > lat - 8)
# ]

# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["geometry"].y,
                        lon=Hazard_map["geometry"].x,
                        color="World Protected Area Vulnerability",
                        hover_data="World Protected Area Vulnerability",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

fig_name_shoreline=Hazard_map.columns[-1]+' shoreline_plot-RAW.png'
pio.write_image(fig, os.path.join(directory_path, fig_name_shoreline))